### 库

In [2]:
import base64
from time import sleep, time
import pymysql

In [3]:
from driverOps import getDriver, waitTillOpen, waitTillLoaded, closeAllOther
from AminerScholarExtracter import AminerScholarExtracter
from AminerSearchExtracter import searchName, getScholarPage, getScholarIds
from utils import stdAminerId, typeOfScript
from driverUtils import getTextByXpath, getTextsByXpath, getAttributeByXpath, getTextById
from cfg import tqdmNcols as ncols, mysqlIp, mysqlPort, mysqlUser, mysqlPassword, mysqlDb, waitUnit, waitLoading, scholarGap
from MysqlInserter import addCursor, checkScholar, checkDoneName, insertScholar, insertDoneName, select, sql
from MysqlInserter import insertAwards, insertPortrait, insertTalentPools, insertWebpages, insertPapers, insertPublish
from namelist import names

In [4]:
from vUtil.vFile import fprint
from vUtil.vEmail import sendEmail
from vUtil.vTime import getNow, convertSeconds

In [5]:
from PIL import Image
import io
from vVariableInspector import _getshapeof

In [6]:
if typeOfScript() == 'jupyter':
    from tqdm.notebook import tqdm, trange
else:
    from tqdm import tqdm, trange
from time import sleep

### 连接数据库

In [7]:
db = pymysql.connect(mysqlIp, mysqlUser, mysqlPassword,port=mysqlPort,charset='utf8',db=mysqlDb)
cursor = db.cursor()
addCursor(cursor)

In [17]:
select('count(distinct display_name)', 'landinn.authors')

landinn.authors,count(distinct display_name)
0,311987


In [9]:
select('count(*)', 'AminerClimbed.done_name')

AminerClimbed.done_name,count(*)
0,87053


In [18]:
select('count(*)', 'landinn.authors')

landinn.authors,count(*)
0,472944


In [12]:
rst = sql('''
select display_name as names from
(select distinct display_name from landinn.authors) as a
left join
(select name from AminerClimbed.done_name) as b
on a.display_name = b.name where name is null and display_name is not null;
''')

In [17]:
file = 'n500k.py'

In [29]:
def writeNames (i, names):
    file = f'n{i}.py'
    folder = 'n500k'
    fprint('names = [\n', file, folder)
    for name in tqdm(names, leave=False):
        fprint(f'{repr(name)},\n', file, folder)
    fprint(']\n', file, folder)

In [30]:
n = 6
for i in trange(n):
    writeNames (i, rst['names'][i::n])

In [15]:
db.close()